In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

pd.set_option("display.float_format", "{:.2f}".format)
pd.set_option("display.max_columns", 80)
pd.set_option("display.max_rows", 80)

# EDA , Feature Engineering, Feature Selection

In [22]:
# Read the Data
train = pd.read_csv("train_MpHjUjU.csv") 
test = pd.read_csv("test_hXY9mYw.csv")

In [23]:
train.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1


In [24]:
train.isna().sum()
# No NA values found, LastWorkingDate is NA, as long as the employee hasn't resigned

MMM-YY                      0
Emp_ID                      0
Age                         0
Gender                      0
City                        0
Education_Level             0
Salary                      0
Dateofjoining               0
LastWorkingDate         17488
Joining Designation         0
Designation                 0
Total Business Value        0
Quarterly Rating            0
dtype: int64

In [25]:
# Convert to Date Time
train['Dateofjoining'] = pd.to_datetime(train['Dateofjoining'], format='%Y-%m-%d')
train['LastWorkingDate'] = pd.to_datetime(train['LastWorkingDate'], format='%Y-%m-%d')
train['MMM-YY'] = pd.to_datetime(train['MMM-YY'], format='%Y-%m-%d')

In [26]:
def featurize(df):
  ''' 
  This function generates additional features based on Data given
  Returns a dataframe with additional features
  '''

  # Count no. of times Quarterly Rating has changed for an employee
  df.sort_values(['Emp_ID','MMM-YY'],inplace=True)
  df['Rating_chng'] = df.groupby(['Emp_ID'])['Quarterly Rating'].diff().fillna(0)
  rating_change_ = df.groupby('Emp_ID').Rating_chng.nunique().reset_index()
  rating_change_.rename({'Rating_chng': 'Rating_chng_count'}, axis=1, inplace=True)
  rating_change_["Rating_chng_count"] = rating_change_["Rating_chng_count"].apply(lambda x:x-1)
  df['Resigned'] = df["LastWorkingDate"].apply(lambda x:0 if pd.isnull(x) else 1)
  df["Rating Increased"] = df['Rating_chng'].apply(lambda x:1 if (x>0) else 0)
  df["Rating Decreased"] = df['Rating_chng'].apply(lambda x:1 if (x<0) else 0)

  # Calculate the number of times designation has changed for an employee
  df["designation_changed"] = df["Designation"] - df["Joining Designation"]
  designation_change_ = df.groupby(['Emp_ID','designation_changed']).size().reset_index(name='count')
  designation_change_ = designation_change_.groupby('Emp_ID')["designation_changed"].apply(lambda x:x[x>0].count()).reset_index() 

  #Generate Quarter and Year from Reporting Date
  df['Reporting_quarter'] = df['MMM-YY'].dt.quarter
  df['Reporting_year'] = df['MMM-YY'].dt.year

  #Times an employee did no business
  zeroBusiness = df.groupby('Emp_ID')['Total Business Value'].apply(lambda x: x[x == 0].count()).reset_index()
  zeroBusiness.rename({'Total Business Value': 'Zero_Business_freq'}, axis=1, inplace=True)

  #Times the generated business had to be pulled off/cancelled
  negativeBusiness = df.groupby('Emp_ID')['Total Business Value'].apply(lambda x: x[x < 0].count()).reset_index()
  negativeBusiness.rename({'Total Business Value': 'Negative_Business_freq'}, axis=1, inplace=True)

  #Total Business Value Generated
  totalBusiness = df.groupby('Emp_ID')['Total Business Value'].apply(lambda x: x.sum()).reset_index()
  totalBusiness.rename({'Total Business Value': 'total_business_lifetime'}, axis=1, inplace=True)

  #Times Quarter on Quarter Business Value has dropped
  df["period"] = df["Reporting_year"].astype(str) +"_"+ df["Reporting_quarter"].astype(str)
  quarterlyBusiness = df.groupby(['Emp_ID','period'])['Total Business Value'].apply(lambda x: x.sum()).reset_index()
  quarterlyBusiness.rename({'Total Business Value': 'total_business_year_quarter'}, axis=1, inplace=True)
  quarterlyBusiness['pct_ch_tbv_period'] = (quarterlyBusiness.groupby('Emp_ID')['total_business_year_quarter'].apply(pd.Series.pct_change)).fillna(0)
  businessdropped_quarters = quarterlyBusiness.groupby('Emp_ID')['total_business_year_quarter'].apply(lambda x: x[x < 0].count()).reset_index()
  businessdropped_quarters.rename({'total_business_year_quarter': 'business_quarters_dropped'}, axis=1, inplace=True)

  #Days Worked
  df['Days Worked'] = (df['LastWorkingDate'] - df['Dateofjoining']).dt.days.fillna(0)

  #Merge all
  rating_change_= rating_change_.join(zeroBusiness["Zero_Business_freq"]).join(negativeBusiness["Negative_Business_freq"])
  rating_change_ = rating_change_.join(totalBusiness["total_business_lifetime"]).join(businessdropped_quarters["business_quarters_dropped"]).join(designation_change_["designation_changed"])
  features = df.groupby('Emp_ID')['Rating Increased','Resigned','Salary','Days Worked'].sum().reset_index()
  df_train = pd.merge(rating_change_, features, on="Emp_ID")
  df_train = pd.merge(df_train, df.groupby("Emp_ID")["Age","Gender","Dateofjoining",'City', 'Education_Level',"Designation"].max().reset_index(), on="Emp_ID")

  return df_train

In [27]:
# Generate Additional Features
df_train = featurize(train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [28]:
df_train.head()

,Emp_ID,Rating_chng_count,Zero_Business_freq,Negative_Business_freq,total_business_lifetime,business_quarters_dropped,designation_changed,Rating Increased,Resigned,Salary,Days Worked,Age,Gender,Dateofjoining,City,Education_Level,Designation
0,1,0,1,1,1715580,0,0,0,1,172161,78.00,28,Male,2015-12-24,C23,Master,1
1,2,0,2,0,0,0,0,0,0,134032,0.00,31,Male,2017-11-06,C7,Master,2
2,4,0,4,0,350000,0,0,0,1,328015,141.00,43,Male,2016-12-07,C13,Master,2
3,5,0,2,0,120360,0,0,0,1,139104,58.00,29,Male,2016-01-09,C9,College,1
4,6,1,4,0,1265000,0,0,1,0,393640,0.00,31,Female,2017-07-31,C11,Bachelor,3


In [29]:
# Calculate total no. of days worked as of 30th June, 2018(End of Quarter2 of 2018)
df_train.rename({'Days Worked': 'Days_Worked'}, axis=1, inplace=True)
run_date = pd.to_datetime("2018-06-30",format='%Y-%m-%d')
df_train["Days_Worked"] = df_train.apply(lambda x: (run_date-x.Dateofjoining).days if x.Days_Worked ==0 else x.Days_Worked,axis=1)

In [71]:
df_training = df_train.loc[~df_train["Emp_ID"].isin(test["Emp_ID"])]
df_test = df_train.loc[df_train["Emp_ID"].isin(test["Emp_ID"])]

COX PH FITTER MODEL

In [30]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

columns_to_scale  = ['Rating_chng_count', 'Zero_Business_freq', 'Negative_Business_freq','total_business_lifetime', 'business_quarters_dropped','designation_changed', 'Rating Increased', 'Salary', 'Designation']

# Instantiate encoder/scaler
scaler = StandardScaler()

# Scale and Encode Separate Columns
scaled_columns  = scaler.fit_transform(df_train[columns_to_scale]) 
X_train_scale = pd.DataFrame(scaled_columns,columns=columns_to_scale)

#Generate Dummies
df_dummy = pd.get_dummies(df_train[['Gender', 'City', 'Education_Level']], drop_first=True)
X_train_scale = X_train_scale.join(df_dummy).join(df_train["Resigned"]).join(df_train["Days_Worked"])


In [32]:
!pip install lifelines
from lifelines import CoxPHFitter
# Using Cox Proportional Hazards model
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
cph.fit(X_train_scale, 'Days_Worked', event_col='Resigned')   ## Fit the data to train the model
cph.print_summary()    ## HAve a look at the significance of the features

     |████████████████████████████████| 348 kB 5.0 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4049 sha256=b84866ce3f3ea35147e1b7e68954e9ef130ef27f372636b401ad3ad9ce20497d
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma


<lifelines.CoxPHFitter: fitted with 2381 total observations, 765 right-censored observations>
             duration col = 'Days_Worked'
                event col = 'Resigned'
      baseline estimation = breslow
   number of observations = 2381
number of events observed = 1616
   partial log-likelihood = -10504.55
         time fit was run = 2021-11-22 12:27:47 UTC

---
                            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                         
Rating_chng_count          -0.05       0.96       0.06            -0.16             0.07                 0.85                 1.07
Zero_Business_freq          0.16       1.18       0.03             0.11             0.21                 1.11                 1.24
Negative_Business_freq      0.01       1.01       0.03            -0.05             0.07                 0.95                 1.07
total_business_lifetime    -0.55       0.58       0.11            -0.77            -0.33                 0.46                 0.72
business_quarters_dropped   0.02       1.02       0.03            -0.03             0.07                 0.97                 1.07
designation_changed        -0.37       0.69       0.04            -0.45            -0.29                 0.64                 0.75
Rating Increased           -0.41       0.66       0.06            -0.53            -0.29                 0.59                 0.75
Salary                     -0.68       0.51       0.09            -0.85            -0.51                 0.43                 0.60
Designation                -0.03       0.97       0.03            -0.09             0.03                 0.91                 1.03
Gender_Male                -0.05       0.95       0.05            -0.15             0.05                 0.86                 1.05
City_C10                    0.23       1.26       0.19            -0.14             0.59                 0.87                 1.81
City_C11                    0.24       1.27       0.20            -0.15             0.64                 0.86                 1.89
City_C12                    0.30       1.35       0.19            -0.08             0.68                 0.93                 1.97
City_C13                    0.31       1.36       0.19            -0.06             0.68                 0.94                 1.98
City_C14                   -0.01       0.99       0.19            -0.38             0.36                 0.68                 1.44
City_C15                    0.11       1.12       0.18            -0.24             0.47                 0.79                 1.60
City_C16                   -0.08       0.92       0.20            -0.46             0.30                 0.63                 1.36
City_C17                    0.10       1.10       0.19            -0.28             0.47                 0.76                 1.61
City_C18                   -0.02       0.98       0.20            -0.41             0.38                 0.66                 1.46
City_C19                   -0.06       0.94       0.21            -0.47             0.34                 0.63                 1.41
City_C2                     0.18       1.19       0.19            -0.20             0.55                 0.82                 1.74
City_C20                    0.07       1.08       0.17            -0.25             0.40                 0.78                 1.49
City_C21                   -0.09       0.91       0.20            -0.48             0.30                 0.62                 1.34
City_C22                    0.04       1.04       0.20            -0.35             0.42                 0.70                 1.52
City_C23                    0.30       1.35       0.19            -0.07             0.67                 0.93                 1.96
City_C24                    0.38       1.46       0.19            -0.00             0.76   

In [33]:
X_train_scale_significant = X_train_scale[['Days_Worked','Resigned','Zero_Business_freq','total_business_lifetime','designation_changed','Rating Increased','Salary']] # Select only significant features
cph_s = CoxPHFitter()   ## Instantiate the class to create a cph object
cph_s.fit(X_train_scale_significant, 'Days_Worked', event_col='Resigned')   ## Fit the data to train the model
cph_s.print_summary()    ## HAve a look at the significance of the features

<lifelines.CoxPHFitter: fitted with 2381 total observations, 765 right-censored observations>
             duration col = 'Days_Worked'
                event col = 'Resigned'
      baseline estimation = breslow
   number of observations = 2381
number of events observed = 1616
   partial log-likelihood = -10521.14
         time fit was run = 2021-11-22 12:46:30 UTC

---
                          coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                       
Zero_Business_freq        0.15       1.16       0.03             0.10             0.20                 1.10                 1.22
total_business_lifetime  -0.57       0.56       0.10            -0.77            -0.37                 0.46                 0.69
designation_changed      -0.37       0.69       0.04            -0.44            -0.29                 0.64                 0.75
Rating Increased         -0.43       0.65       0.05            -0.52            -0.34                 0.59                 0.71
Salary                   -0.68       0.50       0.07            -0.83            -0.54                 0.44                 0.58

                            z      p   -log2(p)
covariate                                      
Zero_Business_freq       5.62 <0.005      25.62
total_business_lifetime -5.61 <0.005      25.52
designation_changed     -9.75 <0.005      72.24
Rating Increased        -9.15 <0.005      63.98
Salary                  -9.21 <0.005      64.69
---
Concordance = 0.79
Partial AIC = 21052.27
log-likelihood ratio test = 1707.93 on 5 df
-log2(p) of ll-ratio test = inf

In [35]:
pred = cph_s.predict_survival_function(X_train_scale_significant)

In [66]:
resdf = pd.DataFrame()
resdf["Emp_ID"]=df_train["Emp_ID"]
resdf["Survival_Proba"]=pd.Series([pred.loc[X_train_scale_significant.loc[i,"Days_Worked"],i] for i in range(X_train_scale_significant.shape[0])])
#X_train_scale_significant["Surviavl_Proba"] = X_train_scale_significant.apply(lambda x:x.index)

In [68]:
resdf["Target"] = resdf["Survival_Proba"].apply(lambda x:0 if (x>0.5) else 1)

In [72]:
submission = resdf.loc[resdf["Emp_ID"].isin(test["Emp_ID"])]
del submission["Emp_ID"]
submission.to_csv('submission_cox.csv', index=False)

Treating the Problem as a Classification Problem : LGBM,XGBOOST, RF and a Voting Classifier consisting of all 3

In [ ]:
scaled_columns  = scaler.transform(X_test[columns_to_scale]) 
encoded_columns =    ohe.transform(X_test[columns_to_encode])
X_test_std = np.concatenate([scaled_columns, encoded_columns], axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score

def evaluate(model, X_train, X_test, y_train, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    #print(y_test_pred)
    #print(y_test)

    print("TRAINIG RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_train, y_train_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_train, y_train_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

    print("TESTING RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_test_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(solver='liblinear', penalty='l1')
lr_clf.fit(X_train_std, y_train)
evaluate(lr_clf, X_train_std, X_test_std, y_train, y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 263  272]
 [  67 1064]]
ACCURACY SCORE:
0.7965
CLASSIFICATION REPORT:
               0       1  accuracy  macro avg  weighted avg
precision   0.80    0.80      0.80       0.80          0.80
recall      0.49    0.94      0.80       0.72          0.80
f1-score    0.61    0.86      0.80       0.74          0.78
support   535.00 1131.00      0.80    1666.00       1666.00
TESTING RESULTS: 
CONFUSION MATRIX:
[[116 114]
 [ 24 461]]
ACCURACY SCORE:
0.8070
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision   0.83   0.80      0.81       0.82          0.81
recall      0.50   0.95      0.81       0.73          0.81
f1-score    0.63   0.87      0.81       0.75          0.79
support   230.00 485.00      0.81     715.00        715.00


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, bootstrap=False,
#                                      class_weight={0:stay, 1:leave}
                                    )
rf_clf.fit(X_train_std, y_train)
evaluate(rf_clf, X_train_std, X_test_std, y_train, y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 535    0]
 [   0 1131]]
ACCURACY SCORE:
1.0000
CLASSIFICATION REPORT:
               0       1  accuracy  macro avg  weighted avg
precision   1.00    1.00      1.00       1.00          1.00
recall      1.00    1.00      1.00       1.00          1.00
f1-score    1.00    1.00      1.00       1.00          1.00
support   535.00 1131.00      1.00    1666.00       1666.00
TESTING RESULTS: 
CONFUSION MATRIX:
[[150  80]
 [ 49 436]]
ACCURACY SCORE:
0.8196
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision   0.75   0.84      0.82       0.80          0.82
recall      0.65   0.90      0.82       0.78          0.82
f1-score    0.70   0.87      0.82       0.79          0.82
support   230.00 485.00      0.82     715.00        715.00


In [ ]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
xgb_clf.fit(X_train_std, y_train)

evaluate(xgb_clf, X_train_std, X_test_std, y_train, y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 364  171]
 [  55 1076]]
ACCURACY SCORE:
0.8643
CLASSIFICATION REPORT:
               0       1  accuracy  macro avg  weighted avg
precision   0.87    0.86      0.86       0.87          0.86
recall      0.68    0.95      0.86       0.82          0.86
f1-score    0.76    0.90      0.86       0.83          0.86
support   535.00 1131.00      0.86    1666.00       1666.00
TESTING RESULTS: 
CONFUSION MATRIX:
[[144  86]
 [ 32 453]]
ACCURACY SCORE:
0.8350
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision   0.82   0.84      0.83       0.83          0.83
recall      0.63   0.93      0.83       0.78          0.83
f1-score    0.71   0.88      0.83       0.80          0.83
support   230.00 485.00      0.83     715.00        715.00


In [ ]:
from lightgbm import LGBMClassifier

lgb_clf = LGBMClassifier()
lgb_clf.fit(X_train_std, y_train)

evaluate(lgb_clf, X_train_std, X_test_std, y_train, y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 523   12]
 [   8 1123]]
ACCURACY SCORE:
0.9880
CLASSIFICATION REPORT:
               0       1  accuracy  macro avg  weighted avg
precision   0.98    0.99      0.99       0.99          0.99
recall      0.98    0.99      0.99       0.99          0.99
f1-score    0.98    0.99      0.99       0.99          0.99
support   535.00 1131.00      0.99    1666.00       1666.00
TESTING RESULTS: 
CONFUSION MATRIX:
[[165  65]
 [ 48 437]]
ACCURACY SCORE:
0.8420
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision   0.77   0.87      0.84       0.82          0.84
recall      0.72   0.90      0.84       0.81          0.84
f1-score    0.74   0.89      0.84       0.82          0.84
support   230.00 485.00      0.84     715.00        715.00


In [ ]:
from catboost import CatBoostClassifier

cb_clf = CatBoostClassifier()
cb_clf.fit(X_train_std, y_train, verbose=0)

evaluate(cb_clf, X_train_std, X_test_std, y_train, y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[ 444   91]
 [  28 1103]]
ACCURACY SCORE:
0.9286
CLASSIFICATION REPORT:
               0       1  accuracy  macro avg  weighted avg
precision   0.94    0.92      0.93       0.93          0.93
recall      0.83    0.98      0.93       0.90          0.93
f1-score    0.88    0.95      0.93       0.92          0.93
support   535.00 1131.00      0.93    1666.00       1666.00
TESTING RESULTS: 
CONFUSION MATRIX:
[[158  72]
 [ 36 449]]
ACCURACY SCORE:
0.8490
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision   0.81   0.86      0.85       0.84          0.85
recall      0.69   0.93      0.85       0.81          0.85
f1-score    0.75   0.89      0.85       0.82          0.85
support   230.00 485.00      0.85     715.00        715.00


In [ ]:
from sklearn.ensemble import  VotingClassifier
eclf1 = VotingClassifier(estimators=[('lgb', lgb_clf), ('xg', xgb_clf), ('cat', cb_clf)], voting='soft')
#X_train_fs,X_test_fs = select_features(res, Y_train, res2,150)
eclf1 = eclf1.fit(X_train_std, y_train)
evaluate(eclf1, X_train_std, X_test_std, y_train, y_test)

Learning rate set to 0.012811
0:	learn: 0.6881743	total: 1.85ms	remaining: 1.85s
1:	learn: 0.6821818	total: 3.7ms	remaining: 1.85s
2:	learn: 0.6762631	total: 5.64ms	remaining: 1.88s
3:	learn: 0.6703039	total: 7.7ms	remaining: 1.92s
4:	learn: 0.6652059	total: 9.64ms	remaining: 1.92s
5:	learn: 0.6614203	total: 11ms	remaining: 1.83s
6:	learn: 0.6579530	total: 12.9ms	remaining: 1.83s
7:	learn: 0.6543510	total: 14.8ms	remaining: 1.83s
8:	learn: 0.6499291	total: 16.6ms	remaining: 1.83s
9:	learn: 0.6449747	total: 18.4ms	remaining: 1.82s
10:	learn: 0.6391673	total: 20.3ms	remaining: 1.82s
11:	learn: 0.6348925	total: 22.1ms	remaining: 1.82s
12:	learn: 0.6300162	total: 23.9ms	remaining: 1.81s
13:	learn: 0.6261047	total: 25.7ms	remaining: 1.81s
14:	learn: 0.6226492	total: 27.7ms	remaining: 1.82s
15:	learn: 0.6187853	total: 29.6ms	remaining: 1.82s
16:	learn: 0.6146922	total: 31.5ms	remaining: 1.82s
17:	learn: 0.6100907	total: 33.2ms	remaining: 1.81s
18:	learn: 0.6052225	total: 35.4ms	remaining: 1.

In [ ]:
y_test_pred = eclf1.predict(X_test_std)
y_train_pred = eclf1.predict(X_train_std)

In [ ]:
pd.DataFrame(y_train_pred).to_csv("Train_pred.csv")
pd.DataFrame(y_test_pred).to_csv("Test_pred.csv")